# KPI Calculation


#### Data preperation

In [ ]:
import pandas as pd
import numpy as np

df_boston = pd.read_csv('./data/boston_2015.csv', dtype={'start_station_id': np.int64, 'end_station_id': 'string', 'end_station_name': 'string', 'start_station_name': 'string', 'bike_id': np.int64, 'user_type': 'string'})
df_boston['end_station_id'].unique()
df_boston.drop(index=df_boston.loc[df_boston["end_station_id"] == "\\N"].index, inplace=True, axis=1)
df_boston = df_boston.astype({'end_station_id': np.int64})
df_boston['start_time'] = pd.to_datetime(df_boston['start_time'], format='%Y-%m-%d %X')
df_boston['end_time'] = pd.to_datetime(df_boston['end_time'], format='%Y-%m-%d %X')
len(df_boston)

### Utilization
#### Usage Rate (travel time / total possible travel time)

In [ ]:
from datetime import datetime, timedelta
amountOfBikes = len(df_boston['bike_id'].unique())

possibleTravelTime = amountOfBikes*365*24*60
possibleTravelTime = timedelta(minutes=possibleTravelTime)

df_boston['trip_length'] = (df_boston['end_time'] - df_boston['start_time'])
realTravelTime = df_boston['trip_length'].sum()

usageRate = realTravelTime.total_seconds() / possibleTravelTime.total_seconds()
print(usageRate*100)

In [37]:

df_usage_rate = df_boston.copy()
df_usage_rate.dtypes
# df_usage_rate['Weekday'] = df_usage_rate['start_time'].dt.day_name()
# df_usage_rate.loc[df_usage_rate['start_time'] < datetime(2015,1,2)].head()
# df_usage_rate['start_hour'] = df_usage_rate['start_time'].dt.hour
# df_usage_rate['end_hour'] = df_usage_rate['end_time'].dt.hour
# df_usage_rate_overlapping = df_usage_rate.loc[df_usage_rate['start_hour'] != df_usage_rate['end_hour']]
df_usage_rate['times_tmp'] = df_usage_rate.apply((lambda x: [x.start_time.ceil('H')- timedelta(seconds=1), x.start_time.ceil('H')] if x.start_time.hour != x.end_time.hour else None), axis=1)
# df_usage_rate.apply(lambda x: print(x.start_time), axis=1)
# df_usage_rate.set_index('start_time')
# df_usage_rate[df_usage_rate['start_time']=='2015-01-01 01:28:27']



#df_rentals_month_weekday= df_usage_rate.groupby(['Month', 'Weekday']).size().reset_index(name='num_rides')

#df_rentals_month_weekday.Weekday = pd.Categorical(df_rentals_month_weekday.Weekday,categories=days)
#df_rentals_month_weekday = df_rentals_month_weekday.sort_values('Weekday')

#df_rentals_month_weekday.Month = pd.Categorical(df_rentals_month_weekday.Month,categories=months)
#df_rentals_month_weekday = df_rentals_month_weekday.sort_values('Month')

#sns.lineplot(data=df_rentals_month_weekday, x='Weekday', y='num_rides', hue='Month')
df_usage_rate['start_time']

0         2015-01-01 00:21:44
1         2015-01-01 00:27:03
2         2015-01-01 00:31:31
3         2015-01-01 00:53:46
4         2015-01-01 01:07:06
                  ...        
1122553   2015-12-31 23:35:04
1122554   2015-12-31 23:37:29
1122555   2015-12-31 23:38:16
1122556   2015-12-31 23:46:04
1122557   2015-12-31 23:58:18
Name: start_time, Length: 1122557, dtype: datetime64[ns]

In [38]:
df_usage_rate=df_usage_rate.explode('times_tmp')
df_usage_rate['end_time'] = df_usage_rate.apply(lambda x: x.times_tmp if x.times_tmp.minute == 59 else x.end_time, axis=1)
df_usage_rate['start_time'] = df_usage_rate.apply(lambda x: x.times_tmp if x.times_tmp.minute == 0 else x.start_time, axis=1)

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,trip_length,times_tmp
0,2015-01-01 00:21:44,2015-01-01 00:30:47,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,277,Subscriber,0 days 00:09:03,NaT
1,2015-01-01 00:27:03,2015-01-01 00:34:21,80,95,MIT Stata Center at Vassar St / Main St,Cambridge St - at Columbia St / Webster Ave,648,Subscriber,0 days 00:07:18,NaT
2,2015-01-01 00:31:31,2015-01-01 00:35:46,91,68,One Kendall Square at Hampshire St / Portland St,Central Square at Mass Ave / Essex St,555,Subscriber,0 days 00:04:15,NaT
3,2015-01-01 00:53:46,2015-01-01 00:59:59,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,1307,Subscriber,0 days 00:07:12,2015-01-01 00:59:59
3,2015-01-01 01:00:00,2015-01-01 01:00:58,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,1307,Subscriber,0 days 00:07:12,2015-01-01 01:00:00
...,...,...,...,...,...,...,...,...,...,...
1122554,2015-12-31 23:37:29,2015-12-31 23:47:36,135,43,ID Building East,Rowes Wharf - Atlantic Ave,649,Subscriber,0 days 00:10:07,NaT
1122555,2015-12-31 23:38:16,2015-12-31 23:47:26,135,43,ID Building East,Rowes Wharf - Atlantic Ave,1552,Subscriber,0 days 00:09:10,NaT
1122556,2015-12-31 23:46:04,2015-12-31 23:48:04,36,36,Boston Public Library - 700 Boylston St.,Boston Public Library - 700 Boylston St.,1083,Customer,0 days 00:02:00,NaT
1122557,2015-12-31 23:58:18,2015-12-31 23:59:59,105,6,Lower Cambridgeport at Magazine St/Riverside Rd,Cambridge St. at Joy St.,1305,Subscriber,0 days 00:14:36,2015-12-31 23:59:59


In [40]:
df_usage_rate['start_time'][8].round('60min') - timedelta(seconds=1)

Timestamp('2015-01-01 04:59:59')

### Coverage

### Revenue

### Clustering